#### 导入相关包

In [2]:
from used_packages import *

## 一、 读取数据
### 1、设置数据路径

In [3]:
data_path = r'H:\Solomon\EEG_WORK\data\feature\ctm_after_rhythm_without_processing'
result_path = r'H:\Solomon\EEG_WORK\result\0125_rhythm_ctm_result_without_process'
os.makedirs(result_path)

### 2 读取ctm特征数据

In [4]:
def read_ctm(r=1):
    rhythm_list = ['delta', 'theta', 'alpha', 'beta', 'gamma']
    # imf_list = ['imf_1', 'imf_2', 'imf_3', 'imf_4']

    df = pd.DataFrame()

    for rhythm in rhythm_list:
        feature_path = f"{data_path}\\r={r}\\{rhythm}"
        tmp_df = pd.DataFrame()
        for file in os.listdir(feature_path):
            tmp = pd.read_csv(f"{feature_path}\\{file}", index_col=0)
            pid = file[11:13]
            tmp['pid'] = pid
            tmp_df = tmp_df.append(tmp)
        tmp_df['rhythm'] = rhythm
        df = df.append(tmp_df)
    return df
df = read_ctm()
df

,Fp1,Fp2,F7,F3,Fz,F4,F8,FC5,FC1,FC2,...,PPO2h,PPO6h,PPO10h,P10,I1,OI1h,OI2h,I2,pid,rhythm
C12,715,941,462,688,840,692,631,392,1017,1479,...,683,414,585,710,626,588,592,689,01,delta
C11,556,735,180,601,693,797,650,294,959,706,...,381,465,337,431,451,514,533,553,01,delta
C12,1223,1340,775,1181,722,860,805,528,823,796,...,722,561,499,646,647,584,542,577,01,delta
C12,378,619,410,323,483,559,491,271,925,1543,...,399,248,275,270,207,206,212,223,01,delta
C11,657,832,511,700,614,604,529,562,1168,1200,...,590,813,622,530,398,431,475,505,01,delta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C21,1634,1868,1150,1724,2315,1603,1394,1182,2521,2645,...,1048,693,844,1018,637,633,726,814,20,gamma
C21,1650,1834,1094,1652,2359,1605,1412,1214,2476,2815,...,987,594,802,962,633,643,789,852,20,gamma
C21,1676,1718,1052,1534,2312,1439,1549,1255,2422,2713,...,1106,579,767,992,630,639,722,774,20,gamma
C21,1706,1762,1213,1670,2474,1465,1499,1215,2484,2665,...,1145,534,676,898,727,679,708,732,20,gamma


## 分类实验
#### 一些分类参数

In [12]:
# 半径情况
r_list = [1,2,10]

# 节律情况
rhythm_list = ['delta', 'theta', 'alpha', 'beta', 'gamma']
# imf_list = ['imf_1', 'imf_2', 'imf_3', 'imf_4']


# 通道选择情况
channel_need = ['FFC1h', 'AFF2h', 'FCC4h', 'TTP8h', 'FCC1h', 'F1',
                'FT10', 'C5', 'FFT7h', 'F3', 'AFp2']

# 标签转换，C2 为大类，C4 为小类
# cls_label_trans = {'C2':{'C11': 1, 'C12': 1, 'C21': 2, 'C22': 2},
#                    'C4':{'C11': 1, 'C12': 2, 'C21': 3, 'C22': 4}}

cls_label_trans = {'C2':{'C11': 1, 'C12': 1, 'C21': -1, 'C22': -1},
                   'C41':{'C11': 1, 'C12': -1},
                   'C42':{'C21': 1, 'C22': -1}}

# 内外的交叉验证
inner_cv = KFold(n_splits=3, shuffle=True, random_state=99)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=1)

# 分类器
clf = svm.SVC()

# 参数矩阵
param_grid = [
                # {'C': [1, 10, 100],
               # 'kernel': ['linear']},
              {'C': [1, 10, 100],
               'kernel': ['rbf'],
               'gamma': [1e-3, 1e-2, 1e-1]}
              ]

#### ④ 单人5通道单节律分类
##### ④.① Prefronta Cortex

In [17]:
param_grid = [
                # {'C': [1, 10, 100],
               # 'kernel': ['linear']},
              {'C': [1, 10, 100, 1000],
               'kernel': ['rbf'],
               'gamma': [1e-3, 1e-2, 1e-1]}
              ]

Prefrontal_Cortex = ['Fpz','Fp1','Fp2','AFp1', 'AFp2']
# 按照半径展开
for r in r_list:
    df = read_ctm(r)

    # 按照 C2 和 c4 展开分类
    for cls_key in cls_label_trans.keys():
        ans_df = pd.DataFrame(columns=rhythm_list, index=range(1,21))
        # 按照节律展开
        for rhythm in rhythm_list:
            # 按照pid 展开
            for pid in range(1, 21):
                sys.stdout.write(f"\r class {cls_key} rhythm {rhythm} pid {pid}")
                X_Y_data = df[df.index.isin(cls_label_trans[cls_key].keys())]

                X_Y_data = X_Y_data.loc[X_Y_data.pid==f'{pid:02}']
                X_Y_data = X_Y_data.loc[X_Y_data.rhythm == rhythm].drop(['pid', 'rhythm'], axis=1)
                Y = np.array([cls_label_trans[cls_key][i] for i in X_Y_data.index])

                X = X_Y_data[Prefrontal_Cortex].values
                gs = GridSearchCV(estimator=clf, param_grid=param_grid, iid=False,scoring='accuracy', cv=inner_cv, n_jobs=8)
                score = cross_val_score(gs, X, Y, cv=outer_cv, scoring='accuracy')
                ans_df.loc[pid, rhythm] = f"{np.mean(score) * 100:2.1f}%±{np.std(score, ddof=1) * 100:2.1f}%"

        ans_df.to_csv(f"{result_path}\\result4_20people_Prefronta Cortex_{cls_key}_r{r}.csv", encoding="GBK")

 class C42 rhythm gamma pid 20

##### ④.② Visual Cortex

In [18]:
param_grid = [
                # {'C': [1, 10, 100],
               # 'kernel': ['linear']},
              {'C': [1, 10, 100, 1000],
               'kernel': ['rbf'],
               'gamma': [1e-3, 1e-2, 1e-1]}
              ]

visual_Cortex = ['Oz','O1','O2','OI1h', 'OI2h']
# 按照半径展开
for r in r_list:
    df = read_ctm(r)

    # 按照 C2 和 c4 展开分类
    for cls_key in cls_label_trans.keys():
        ans_df = pd.DataFrame(columns=rhythm_list, index=range(1,21))
        # 按照节律展开
        for rhythm in rhythm_list:
            # 按照pid 展开
            for pid in range(1, 21):
                sys.stdout.write(f"\r class {cls_key} rhythm {rhythm} pid {pid}")

                X_Y_data = df[df.index.isin(cls_label_trans[cls_key].keys())]

                X_Y_data = X_Y_data.loc[X_Y_data.pid==f'{pid:02}']
                X_Y_data = X_Y_data.loc[X_Y_data.rhythm == rhythm].drop(['pid', 'rhythm'], axis=1)
                Y = np.array([cls_label_trans[cls_key][i] for i in X_Y_data.index])

                X = X_Y_data[visual_Cortex].values
                gs = GridSearchCV(estimator=clf, param_grid=param_grid, iid=False,scoring='accuracy', cv=inner_cv, n_jobs=8)
                score = cross_val_score(gs, X, Y, cv=outer_cv, scoring='accuracy')
                ans_df.loc[pid, rhythm] = f"{np.mean(score) * 100:2.1f}%±{np.std(score, ddof=1) * 100:2.1f}%"

        ans_df.to_csv(f"{result_path}\\result4_20people_Visual Cortex_{cls_key}_r{r}.csv", encoding="GBK")
        print(ans_df)

 class C2 rhythm gamma pid 20          delta        theta        alpha         beta        gamma
1    47.7%±6.0%   43.7%±8.2%  48.5%±12.0%   69.8%±3.5%   84.2%±7.9%
2   57.2%±12.8%  58.7%±13.0%  57.2%±11.2%  57.2%±12.8%  69.5%±14.0%
3    48.5%±8.8%  57.4%±11.9%  67.1%±10.4%   81.3%±8.1%   83.5%±7.1%
4   55.5%±12.9%   52.2%±9.7%   51.4%±9.3%  54.7%±13.2%  58.0%±10.5%
5   58.7%±15.0%  45.8%±11.6%  58.0%±13.0%  55.1%±13.7%  60.2%±10.5%
6   56.4%±11.7%  57.1%±11.0%  68.6%±12.5%  76.4%±10.3%   84.3%±7.8%
7    47.7%±4.1%   55.6%±3.1%  58.7%±10.4%   66.1%±7.1%  63.4%±11.0%
8    53.1%±8.0%   53.7%±7.7%   57.1%±7.9%  60.7%±13.6%   57.2%±8.1%
9    47.2%±5.1%   51.4%±5.5%  65.1%±12.4%   48.6%±5.8%   65.3%±8.8%
10   47.6%±4.5%   44.8%±6.0%  58.6%±11.4%  69.0%±14.6%  77.2%±10.5%
11   45.5%±8.1%   45.5%±5.8%   61.9%±8.2%  58.2%±13.2%   49.9%±9.7%
12  55.5%±10.0%   55.5%±5.6%   58.4%±9.0%   64.3%±7.2%  60.6%±11.9%
13   45.0%±5.2%  57.8%±18.1%  48.8%±12.4%  54.8%±11.5%   45.0%±5.2%
14   60.3%±8.5%  63

#### ⑤ 单人5通道多节律分类
##### ⑤.① Prefrontal Cortex

In [15]:
param_grid = [
               #  {'C': [1, 10, 100],
               # 'kernel': ['linear']},
              {'C': [1, 10, 100, 1000],
               'kernel': ['rbf'],
               'gamma': [1e-3, 1e-2, 1e-1]}
              ]

Prefrontal_Cortex = ['Fpz','Fp1','Fp2','AFp1', 'AFp2']
ans_df = pd.DataFrame()
# 按照半径展开
for r in r_list:
    df = read_ctm(r)
    # 按照 C2 和 c4 展开分类
    for cls_key in cls_label_trans.keys():
        # 按照pid 展开
        for pid in range(1, 21):
            sys.stdout.write(f"\r class {cls_key} pid {pid}")

            X_Y_data = df[df.index.isin(cls_label_trans[cls_key].keys())]
            X_Y_data = X_Y_data[visual_Cortex+['rhythm', 'pid']].loc[X_Y_data.pid==f"{pid:02}"].drop(["pid"], axis = 1)
            X_data = pd.DataFrame()
            for rhythm in rhythm_list:

                tmp = X_Y_data.loc[X_Y_data.rhythm == rhythm]
                tmp = tmp.drop('rhythm',axis=1)
                tmp.columns = [f"{i}_{rhythm}" for i in tmp.columns]
                X_data = pd.concat([X_data, tmp], axis=1)
            Y = np.array([cls_label_trans[cls_key][i] for i in X_data.index])
            X = X_data.values

            gs = GridSearchCV(estimator=clf, param_grid=param_grid, iid=False,scoring='accuracy', cv=inner_cv, n_jobs=8)
            score = cross_val_score(gs, X, Y, cv=outer_cv, scoring='accuracy')
            # ans_df.loc[pid, f'r={r}_{cls_key}'] = np.mean(score) * 100
            ans_df.loc[pid, f'r={r}_{cls_key}'] = f"{np.mean(score) * 100:2.1f}%±{np.std(score, ddof=1) * 100:2.1f}%"

        ans_df.to_csv(f"{result_path}\\result4_20people_Prefronta Cortex.csv",encoding="GBK")
print(ans_df)

 class C42 pid 20         r=1_C2      r=1_C41      r=1_C42       r=2_C2      r=2_C41  \
1    92.0%±4.9%  46.2%±10.4%  43.1%±11.7%   96.8%±3.3%  42.8%±14.2%   
2    89.9%±5.3%  74.2%±14.4%  46.7%±12.4%   91.3%±2.0%  67.6%±11.8%   
3    98.5%±2.1%   41.0%±6.6%  55.5%±16.9%  100.0%±0.0%  37.9%±11.2%   
4   84.4%±10.2%  57.5%±14.3%  60.0%±10.4%   77.6%±7.7%  53.4%±15.3%   
5    93.4%±4.8%  42.4%±13.0%   48.7%±9.9%   96.3%±4.5%   53.2%±7.0%   
6    99.3%±1.6%  54.5%±17.1%  58.3%±14.7%  100.0%±0.0%  54.5%±18.1%   
7    87.6%±7.0%  49.3%±11.9%  38.2%±11.9%   93.5%±0.1%   46.6%±9.6%   
8    89.8%±4.2%   49.3%±8.5%  55.7%±11.7%   94.6%±3.0%   40.3%±6.0%   
9    79.9%±7.0%   61.4%±6.4%  47.2%±12.2%   78.5%±7.3%   57.1%±5.1%   
10   99.3%±1.5%   56.0%±7.6%   60.0%±6.4%  100.0%±0.0%  45.3%±17.3%   
11   99.2%±1.7%  55.9%±15.7%   59.1%±6.5%   98.5%±3.4%  51.8%±13.0%   
12   88.3%±4.1%  53.3%±10.9%  45.4%±10.3%   97.1%±3.0%   52.2%±8.7%   
13  100.0%±0.0%  41.5%±12.9%  48.5%±10.5%  100.0%±0.0%  43.1

C:\Users\gxlls\Anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


##### ⑤.② Visual Cortex

In [16]:
param_grid = [
               #  {'C': [1, 10, 100],
               # 'kernel': ['linear']},
              {'C': [1, 10, 100, 1000],
               'kernel': ['rbf'],
               'gamma': [1e-3, 1e-2, 1e-1]}
              ]

visual_Cortex = ['Oz','O1','O2','OI1h', 'OI2h']
ans_df = pd.DataFrame()
# 按照半径展开
for r in r_list:
    df = read_ctm(r)
    # 按照 C2 和 c4 展开分类
    for cls_key in cls_label_trans.keys():
        # 按照pid 展开
        for pid in range(1, 21):
            sys.stdout.write(f"\r class {cls_key} pid {pid}")

            X_Y_data = df[df.index.isin(cls_label_trans[cls_key].keys())]
            X_Y_data = X_Y_data[visual_Cortex+['rhythm', 'pid']].loc[X_Y_data.pid==f"{pid:02}"].drop(["pid"], axis = 1)
            X_data = pd.DataFrame()
            for rhythm in rhythm_list:
                tmp = X_Y_data.loc[X_Y_data.rhythm == rhythm]
                tmp = tmp.drop('rhythm',axis=1)
                tmp.columns = [f"{i}_{rhythm}" for i in tmp.columns]
                X_data = pd.concat([X_data, tmp], axis=1)
            Y = np.array([cls_label_trans[cls_key][i] for i in X_data.index if i in cls_label_trans[cls_key].keys()])
            X = X_data.values

            gs = GridSearchCV(estimator=clf, param_grid=param_grid, iid=False,scoring='accuracy', cv=inner_cv, n_jobs=8)
            score = cross_val_score(gs, X, Y, cv=outer_cv, scoring='accuracy')
            # ans_df.loc[pid, f'r={r}_{cls_key}'] = np.mean(score) * 100
            ans_df.loc[pid, f'r={r}_{cls_key}'] = f"{np.mean(score) * 100:2.1f}%±{np.std(score, ddof=1) * 100:2.1f}%"
        ans_df.to_csv(f"{result_path}\\result4_20people_visual Cortex.csv", encoding="GBK")
print(ans_df)

 class C42 pid 20         r=1_C2      r=1_C41      r=1_C42       r=2_C2      r=2_C41  \
1    92.0%±4.9%  46.2%±10.4%  43.1%±11.7%   96.8%±3.3%  42.8%±14.2%   
2    89.9%±5.3%  74.2%±14.4%  46.7%±12.4%   91.3%±2.0%  67.6%±11.8%   
3    98.5%±2.1%   41.0%±6.6%  55.5%±16.9%  100.0%±0.0%  37.9%±11.2%   
4   84.4%±10.2%  57.5%±14.3%  60.0%±10.4%   77.6%±7.7%  53.4%±15.3%   
5    93.4%±4.8%  42.4%±13.0%   48.7%±9.9%   96.3%±4.5%   53.2%±7.0%   
6    99.3%±1.6%  54.5%±17.1%  58.3%±14.7%  100.0%±0.0%  54.5%±18.1%   
7    87.6%±7.0%  49.3%±11.9%  38.2%±11.9%   93.5%±0.1%   46.6%±9.6%   
8    89.8%±4.2%   49.3%±8.5%  55.7%±11.7%   94.6%±3.0%   40.3%±6.0%   
9    79.9%±7.0%   61.4%±6.4%  47.2%±12.2%   78.5%±7.3%   57.1%±5.1%   
10   99.3%±1.5%   56.0%±7.6%   60.0%±6.4%  100.0%±0.0%  45.3%±17.3%   
11   99.2%±1.7%  55.9%±15.7%   59.1%±6.5%   98.5%±3.4%  51.8%±13.0%   
12   88.3%±4.1%  53.3%±10.9%  45.4%±10.3%   97.1%±3.0%   52.2%±8.7%   
13  100.0%±0.0%  41.5%±12.9%  48.5%±10.5%  100.0%±0.0%  43.1